# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [1]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import nltk, collections, pandas as pd, plotly.express as px
from itertools import chain

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**
 
To begin, download the Gutenberg Corpus, punctuation, and a list of English stopwords. Retrieve the book "Alice's Adventures in Wonderland" from the corpus for further experimentation with collocations and *n*-grams.

In [2]:
_ = nltk.download(['gutenberg', 'punkt', 'stopwords'], quiet=True)
LsStopwords = nltk.corpus.stopwords.words('english')
LsDoc1 = nltk.corpus.gutenberg.words(fileids='carroll-alice.txt')
print(LsDoc1[:30])

['[', 'Alice', "'", 's', 'Adventures', 'in', 'Wonderland', 'by', 'Lewis', 'Carroll', '1865', ']', 'CHAPTER', 'I', '.', 'Down', 'the', 'Rabbit', '-', 'Hole', 'Alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by']


## **$n$-grams**

<span style="color:black">An $n$-gram is a subsequence of $n$ adjacent tokens (i.e., characters, words, or phrases). A lower $n$ tends to produce a higher frequency of similar $n$-grams. For example, in this notebook given $n=2$, you will encounter multiple "*of the*" 2-grams, but fewer "*of the ...*" 2-grams (such as "*of the children's*", "*of the highest*", "*of the event*".  Notably, the longer phrases may not be representative of the underlying text. 
    
<span style="color:black">Create a function `Make_NGrams()` that takes the document `LsDoc` and returns the top `nTop` `n`-grams. You will first use this function to evaluate whether 2-grams are representative of the children's novel.

In [3]:
def Make_NGrams(LsDoc, n=2, nTop=20):
    LToMC = collections.Counter(nltk.ngrams(LsDoc, n=n)).most_common(nTop)  # list of tuples of objects
    df = pd.DataFrame(LToMC, columns=['ngram','Freq'])
    return df.set_index('ngram')

Make_NGrams(LsDoc1, 2).T

ngram,"(,, and)","(., ')","(', t)","(,', said)","(said, the)","(', s)","(,, ')","(', I)","(I, ')","(of, the)","(said, Alice)","(.', ')","(in, a)","(,, I)","(in, the)","(Alice, ,)","(and, the)","(to, the)","(:, ')","(!', said)"
Freq,446,416,216,215,206,195,192,166,135,127,115,110,95,83,76,76,75,69,69,65


<span style="color:black">Notice that the frequency-based search finds frequent, but often meaningless, tokens. Most of the highest frequency tokens are punctuations mixed with stop words. For example, `(', s)` and `(', t)` are drawn from contractions, such as `"he ' s"` and `"can ' t"`. Only `(said, Alice)` and `(Alice, ,)` are likely to be relevant. Thus, the top 2-grams are hardly representative of this book.

<span style="color:black">**Light verbs** (e.g., *say*, *take*, *make*, and *do*) carry little semantic content and several of these words are in your stoplist. You can preprocess the book by removing stop words and keeping only letter words. This drops contractions, words separated by dashes, etc. The goal is to qualitatively evaluate whether this produces a list of 2-grams more relevant to the story in the book.

In [ ]:
sDoc2 = nltk.corpus.gutenberg.raw(fileids='carroll-alice.txt')
LsDoc2 = [s.lower() for s in nltk.word_tokenize(sDoc2) if (not s in LsStopwords) and s.isalpha()]
Make_NGrams(LsDoc2, 2).T

<span style="color:black">Notice that there are four phrases containing `alice` and more phrases containing `king`, `caterpillar` and other distinguished characters from the book. The 2-grams appear more relevant, but it takes a greater familiarity with the textbook to make a meaningful judgment. Domain expertise is an attribute of most unsupervised learning techniques, such as extraction of key phrases that describe the source document. 

## **NLTK Collocations**

<span style="color:black">[Collocations](https://www.nltk.org/howto/collocations.html#collocations) is an expression of commonly co-occuring words. Recall that we could use frequency of occurrence to find representative n-grams. With collocations we can also use metrics that account for word importance. A popular measure is pointwise mutual information ([PMI](https://www.nltk.org/api/nltk.metrics.association.html#module-nltk.metrics.association)), which is a degree of increased information from the additional word in the phrase, regardless of word order:
 
$$\text{PMI}(x,y):=\log\frac{p(x,y)}{p(x)\cdot p(y)}$$
 
<span style="color:black">where $p(x)$ is the probability of the event (or token) $x$ and $p(x,y)$ is the joint probability of events $x,y$. Note that $p$ functions are theoretical probabilities, which, in practice, can be approximated in a multitude of ways (each requiring a set of specific assumptions).
        
You can say that PMI measures the amount of mutual information in its arguments, $x$ and $y$, after their unique information is factored out (through division). A higher PMI suggests a smaller chance of two words coincidentally occuring in the same phrase. PMI and raw frequency counts are available in the `nltk.collocations` library. An implemented approach of estimation of these probabilities can be discovered in [Manning and Schutze 5.3.4](https://wiki.eecs.yorku.ca/course_archive/2014-15/W/6339/_media/chap5.pdf#page=23) and is beyond the scope of this course.

<span style="color:black">The function below takes a document, `LsDoc`, as a list of string words, and some parameters to find a suitable list of collocations. These collocations are packaged as a dataframe and transposed for readability.

In [ ]:
raw_freq = nltk.collocations.BigramAssocMeasures.raw_freq  # a function definition is saved to raw_freq
pmi = nltk.collocations.BigramAssocMeasures.pmi     # pointwise mutual information = log_2(P(x,y)/P(x)/P(y))

def Bigram_Collocations(LsDoc=[], BiGramMeasure=raw_freq, minFreq=1, minChar=1, nTop=50, LsStopwords=[]):
    bcf = nltk.collocations.BigramCollocationFinder.from_words(LsDoc)
    # the filter removes stopwords (regardless of case) or words shorter than minChar
    bcf.apply_word_filter(lambda w: len(w) < minChar or w.lower() in LsStopwords) # min required characters; stopword filter
    bcf.apply_freq_filter(minFreq)                                                # minimal required frequency of collocation
    LTsBGC = bcf.score_ngrams(BiGramMeasure)[:nTop]                               # list of tuples of phrase and score (BiGram collocation)
    df = pd.DataFrame(LTsBGC, columns=['bigram', 'score']).set_index('bigram')
    return df

Bigram_Collocations(LsDoc2, BiGramMeasure=raw_freq).T

<span style="color:black">Unlike frequency-based collocations, PMI-based collocations capture longer and less frequent words. These phrases appear to be unique/more meaningful, but might require some tuning before they are representative.
    
Note that the absolute values of these coefficients are not of importance. You should mostly care about the ordering and the distance between values. Larger PMI indicates greater relevance of the corresponding collocation to the underlying text. So `(said, alice)` appears to be most relevant (with respect to the PMI metric). Notice that `(mock, turtle)` is about half as important as the top keyword.

In [ ]:
Bigram_Collocations(LsDoc2, BiGramMeasure=pmi).round(2).T

<span style="color:black">You can try tuning parameters to find not just unique, but also more representative tuples.

In [ ]:
Bigram_Collocations(LsDoc2, BiGramMeasure=pmi, minFreq=3, minChar=5).round(2).T

<span style="color:black">With the above parameters, notice that there are two collocations containing the name alice and a few more containing characters and objects from the novel, such as turtle, house, hookah, lobster. This is the "better" set of 𝑛-grams produced so far, where "better" correlates with uniqueness and representativeness of the extracted phrases.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**
 
Now, equipped with these concepts and tools, you will tackle a few related tasks.
 
In these optional practice exercises, you will compute the "most relevant" collocations for Jane Austen's "Sense and Sensibility" novel. Some familiarity with the novel (i.e., domain expertise) may be helpful. Here is a list of some characters in the book: *Elinor & Marianne Dashwood, Edward Ferrars, John Willoughby, Colonel Brandon, Henry Dashwood, Margaret Dashwood, John Dashwood, Fanny Dashwood, John Middleton, Lady Middleton, Jennings, Robert Ferrars, Charlotte Palmer, Thomas Palmer, Lucy Steele, Anne Nancy Steele, Harris, Sophia Grey, Morton, Pratt, Eliza Williams, Smith*
 
Preload the names into the `SsNames` set of strings and plot the distribution of name lengths. The longest word has 10 characters and the mean is about 6. You can use this information in setting the desired lengths of the words in collocations.

As you work through these tasks, check your answers by running your code in the `#check solution here` cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

In [ ]:
SsNames = set('Elinor Marianne Dashwood Edward Ferrars John Willoughby Brandon Henry Dashwood \
Margaret Dashwood John Dashwood Fanny Dashwood John Middleton Jennings Robert Ferrars Charlotte Palmer Thomas \
Palmer Lucy Steele Anne Nancy Steele Harris Sophia Grey Morton Pratt Eliza Williams Smith'.lower().split())

print(f'{len(SsNames)} unique names: ', SsNames)
print(f'Longest name is ', max(len(n) for n in SsNames), 'characters')
plot = pd.DataFrame([len(n) for n in SsNames], columns=['Distribution name lengths']).hist(figsize=(20,3));
for ax in plot.flatten():
    ax.set_xlabel('Name length (characters)')
    ax.set_ylabel('Counts')


## Task 1
 
Retrieve `austen-sense.txt` as raw (untokenized) lowercased text from the Gutenberg Corpus and save it to `sDoc3`. Then tokenize it as above, dropping tokens with non-alpha characters and stopwords. Save the resulting list of strings into `LsDoc3`.

<b>Hint:</b> This is similar to the code above. Here is an example of the first few tokens:
        <pre>
        ['sense', 'sensibility', 'jane', 'austen', 'chapter', 'family', ...]
        </pre>

In [ ]:
# check solution here


<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
sDoc3 = nltk.corpus.gutenberg.raw(fileids='austen-sense.txt').lower()
LsDoc3 = [s for s in nltk.word_tokenize(sDoc3) if s.isalpha() and s not in LsStopwords]
print(LsDoc3[:20])
</pre>
</details> 
</font>
<hr>

## Task 2
 
Write a function `BGC_MatchNames(LsDoc=[''], BiGramMeasure=raw_freq, min_freq=1, minChar=1)`, which calls `Bigram_Collocations` (herein BGC) with the same parameters, no stoplist and `nTop=50`. The index of the returned dataframe `df` is flattened (i.e., unnested) into a set of words and only alphabetically sorted intersection between these words and `SsNames` is returned as a list of strings.

The set of novel character names, `SsNames`, allows you to semi-automatically determine the quality of collocation retrieval. The more names are retrieved the better the search quality, assuming all names are equally important in the novel (which, of course, is not the case).

*Note*: flattening a nested container of containers is to unnest all containing containers. For example, `[0,[1],[[2],[3],4],5]` is flattened as `[0,1,2,3,4,5]`.

<b>Hint:</b> To flatten a list of lists, you can write a small double loop or use the <code>itertools.chain</code> function, which returns a generator, which needs to be converted to a set, list or other container object. Here is an example of the first few (alphabetically ordered) matching names:
        <pre>
        ['brandon', 'dashwood', 'edward', 'elinor',...]
        </pre>

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
from itertools import chain
def BGC_MatchNames(LsDoc=['a','b'], SsNames=SsNames, BiGramMeasure=raw_freq, minFreq=1, minChar=1) -> ['']:
    df = Bigram_Collocations(LsDoc, BiGramMeasure=BiGramMeasure, minFreq=minFreq, minChar=minChar, nTop=50)
    LsMatched = sorted(set(chain.from_iterable(df.index)).intersection(SsNames))
    return LsMatched
print(BGC_MatchNames(LsDoc3))
</pre>
</details> 
</font>
<hr>

## Task 3
 
Evaluate the sensitivity of your collocations search to the hyperparameters `minFreq` and `minChar`. Create a double loop of these parameter values ranging from 1 to 10 and for each set of values call `BGC_MatchNames()` with `LsDoc3`, then save the values of hyperparameters and the corresponding length of the returned list of matched novel character names (`nMatches`). This will be a list of tuples, which you can package as a dataframe `df3`. 

Note: The sensitivity to hyperparameters allows us to choose the optimal set of hyperparameters, i.e., such a set that yields the most favorable output of the considered function.

<b>Hint:</b> Here is an example of `df3`'s top few rows:
 
|.|minFreq|minChar|nMatches|
|-|-|-|-|
|0|1|1|11|
|1|1|2|11|
|2|1|3|11|


In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
%time out = [(i,j,len(BGC_MatchNames(LsDoc3, SsNames, raw_freq, i, j))) for i in range(1,10,1) for j in range(1,10,1)]
df3 = pd.DataFrame(out, columns=['minFreq', 'minChar', 'nMatches'])
df3
</pre>
</details> 
</font>
<hr>

## Task 4
 
Now plot `df3` as a scatter plot with `minFreq` as x-axis, `minChar` as y-axis and `nMatches` indicating the bubble size. If done correctly, larger values of hyperparameters will result in fewer matches. Since the longest name in `SsNames` is 10 characters, you will have no match if you restrict your collocation search to at least 11 characters.

<b>Hint:</b> Dataframe object's <code>plot()</code> method has the <code>scatter</code> method with <code>s</code>controlling the size of the bubbles. Your output could look like this (with plotly express library).
        <img src="574_M1_01_task3_scatterplot.png"/> 

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
#_ = df3.plot.scatter(x='minFreq', y='minChar', grid=True, s=df3.nMatches*5); # plot with Matplotlib
sTtl = 'Match counts between words in collocations and words in LsNames'
px.scatter(df3, x='minFreq', y='minChar', color='nMatches', size='nMatches', title=sTtl) # interactive plot with plotly
df3.sort_values('nMatches').T
</pre>The plot is a visualization of sensitivity of collocations (measured via `nMatches`) to `minFreq` and `minChar` hyperparameters. "A picture is worth a thousand words", so a plot may be easier to evaluate than a dataframe (if it is not overplotted is reasonably observable). The chart shows where most matches appear (see legend coloring). The yellowest colors appear at the bottom of the chart indicating that low values of `minChar` (vertical axis) associate with high values of `nMatches` (representing collocations). Collocations do not appear too sensitive to `minFreq`, but lower `minFreq` has greater collocations for higher `minChar`. If we want to produce most collocations, we would choose any value of `minFreq` and <4 values of `minChar`.

</details> 
</font>
<hr>

# Task 5
 
Repeat Task 2 but for a PMI metric. Compare the results of the two metrics. Which one do you prefer and why?

<b>Hint:</b> There is a slight preference towards the PMI metric because it returns 13 matches at its best versus just 12 for the best collocation search with a raw frequency metric. Your output could look like this (with Plotly Express library):
 

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
%time out = [(i,j,len(BGC_MatchNames(LsDoc3, pmi, i, j))) for i in range(1,10,1) for j in range(1,10,1)]
df4 = pd.DataFrame(out, columns=['minFreq', 'minChar', 'nMatches'])
# _ = df4.plot.scatter(x='minFreq', y='minChar', grid=True, s=df4.nMatches*5); plot with Matplotlib
sTtl = 'Match counts between words in collocations and words in LsNames'
px.scatter(df4, x='minFreq', y='minChar', color='nMatches', size='nMatches', title=sTtl) # interactive plot with plotly
df4.sort_values('nMatches').T  # display transposed table (i.e. horizontally)
</pre>
</details> 
</font>
<hr>